In [ ]:
import pandas as pd
import smtplib
import os
import json
import matplotlib.pyplot as plt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders

In [ ]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'ACTUAIZACIÓN: LISTAS DE CLASE E INFORMACIÓN DE GRUPOS - CICLO 2'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = False
ENVIO_A_PENDIENTES = False

if CORREO_TEST:
    RUTA = './INPUT/INFO_CURSOS/TEST.xlsx'
elif ENVIO_A_PENDIENTES:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL_PENDIENTES.xlsx'
else:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL.xlsx'

COLUMNA_CORREO = 'CORREO_DOCENTE'
COLUMNAS = [
    'CODIGO_CURSO',
    'PROFESIONAL+',
    'BACHILLER',
    'TÉCNICO',
    'DESCONOCIDO',
    'TOTAL',
    'INDICE_HOMOGENEIDAD',    
    'NOMBRE_TUTOR',
    'CORREO_TUTOR',
    'TELEFONO_TUTOR',
    'NOMBRE_MENTOR',
    'CORREO_MENTOR',
]

In [ ]:
MSG_HEAD = """<html><body><p>Bucaramanga, 22 de junio de 2022<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022 - Ciclo 2</p>

"""
MSG_BODY = """<p>Atendiendo el asunto, adjunto a este correo se encuentran las listas de clase de el/los grupos en el que usted desempeña el rol de formador. Además, se encuentra la información general de caracterización de cada grupo, junto con los datos de contacto del tutor y el mentor asignado a estos.<br>
La caracterización está dada por el nivel académico de cada tripulante. Existen 4 posibles clases:</p>
<ul>
<li>Bachiller</li>
<li>Técnico</li>
<li>Profesional+</li>
<li>Desconocido</li>
</ul>
<p>A partir de estas 4 clases, se calcula un índice de homogeneidad por cada grupo. Este índice varía de 0 hasta 1, <b>a más cercano esté el índice a 1 significa que el grupo es más homogéneo</b>.<br>
Esta información se le proporciona para que usted tenga una idea general sobre los tripulantes de sus grupos, con el fin de abordar, de ser necesario, diferentes estrategias pedagógicas.<br>
<b>Es fundamental que se pongan en contacto con el tutor y mentor asignado a cada grupo</b>, con el fin de lograr una sinergia y buena comunicación.</p>
"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [ ]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [ ]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

def getCuerpoCorreo(multipart, formador, df): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = ''
    # Contenido adicional que se quiera añadir
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo

def enviarCorreo(origen, destino, asunto, mensaje, adjuntos = []):
    multipart = MIMEMultipart()
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart.attach(MIMEText(mensaje))
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [ ]:
enviados = datos.copy()
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    
    for index, row in dfD.iterrows():
        ruta = './INPUT/INFO_CURSOS/LISTAS/' + row['CODIGO_CURSO'] + '.xlsx'
        adjuntos.append(ruta)
    nombreArchivo = 'INFO_GRUPOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido = getCuerpoCorreo(base, nombreFormador, dfD)
    base = getAdjuntos(base, [], adjuntos)
    try:        
        servidor.sendmail(ORIGEN, destino, base.as_string())
        enviados = enviados[enviados[COLUMNA_CORREO] != destino]
    except:
        enviados.to_excel('./INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL_PENDIENTES.xlsx', index=False)
        print ('Ocurrió un error al enviar el correo a', destino)
        break
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()